In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.logging.set_verbosity(tf.logging.ERROR)

import warnings
warnings.filterwarnings('ignore')

from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder

from aif360.sklearn.preprocessing import Reweighing, ReweighingMeta, FairAdapt, LearnedFairRepresentations
from aif360.sklearn.inprocessing import AdversarialDebiasing, ExponentiatedGradientReduction, GridSearchReduction
from aif360.sklearn.postprocessing import CalibratedEqualizedOdds, RejectOptionClassifier, PostProcessingMeta, RejectOptionClassifierCV
from aif360.sklearn.datasets import fetch_adult
from aif360.sklearn.metrics import disparate_impact_ratio, average_odds_error, generalized_fpr
from aif360.sklearn.metrics import generalized_fnr, difference

# from glob import glob
# import json,os

2023-02-28 12:45:38.595335: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 12:45:38.752314: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-28 12:45:41.092798: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-28 12:45:41.092931: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [2]:
import json,os
from glob import glob
import traceback


def writejson(path,arr,name):
    try:
        os.mkdir(path+'preds/')
    except:
        pass
    f = open(path+'preds/'+name,'w')
    json.dump(arr.tolist(),f)
    f.close()

In [3]:
path = 'data/datasets/credit_tw/'
synthfols = glob("data/datasets/credit_tw/synthetic/*/")
paths = [path]
paths.extend(synthfols)

In [5]:
def task1(path):
    
    if os.path.exists(path+'preds/rocv_pred_average_odds.json') == False:
        print(path)

        train_df = pd.read_csv(path+'train.csv',header=None)

        train_df.columns = ['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
           'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
           'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
           'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
           'DEFAULT_PAY']


        train_df.index = train_df['SEX']

        train_df = train_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

        test_df = pd.read_csv(path+'test.csv',header=None)

        test_df.columns = ['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
           'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
           'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
           'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
           'DEFAULT_PAY']



        test_df.index = test_df['SEX']

        test_df = test_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)



        X_train = train_df.drop(['DEFAULT_PAY'],axis=1)
        X_test = test_df.drop(['DEFAULT_PAY'],axis=1)

        y_train = pd.Series(train_df['DEFAULT_PAY'])
        y_test = pd.Series(test_df['DEFAULT_PAY'])

        y_train = pd.Series(y_train.factorize(sort=True)[0], index=y_train.index)
        y_test = pd.Series(y_test.factorize(sort=True)[0], index=y_test.index)

        X_merged = pd.concat([X_train,X_test])

        ohe = make_column_transformer(
            (OneHotEncoder(sparse=False), X_merged.dtypes == 'object'),
            remainder='passthrough', verbose_feature_names_out=False)

        X_merged_temp  = pd.DataFrame(ohe.fit_transform(X_merged), columns=ohe.get_feature_names_out(), index=X_merged.index)

        X_train  = pd.DataFrame(ohe.transform(X_train), columns=ohe.get_feature_names_out(), index=X_train.index)
        X_test = pd.DataFrame(ohe.transform(X_test), columns=ohe.get_feature_names_out(), index=X_test.index)


        #### BASELINE
        try:
            y_pred_baseline = LogisticRegression(solver='liblinear').fit(X_train, y_train).predict(X_test)
            writejson(path,y_pred_baseline,'baseline_pred.json')    
        except:
            traceback.print_exc()


        #### Adv. Reweighting (Preproc.)

        try:
            rew = ReweighingMeta(estimator=LogisticRegression(solver='liblinear'),
                         reweigher=Reweighing('SEX'))
            params = {'estimator__C': [1, 10]}

            clf = GridSearchCV(rew, params, scoring='accuracy', cv=5)
            clf.fit(X_train, y_train)
            y_pred_advrew = clf.predict(X_test)
            writejson(path,y_pred_advrew,'adv_rew_pred.json')  
        except:
            traceback.print_exc()



        #### LearnedFairRepresentations (Preproc.)

        try:
            LFR = LearnedFairRepresentations(prot_attr='SEX')
            LFR.fit(X_train, y_train)

            y_pred_lfr = LFR.predict(X_test)
            writejson(path,y_pred_lfr,'lfr_pred.json')  
        except:
            traceback.print_exc()

        #### Adv Deb. (Inproc.)

        try:
            adv_deb = AdversarialDebiasing(prot_attr='SEX')
            adv_deb.fit(X_train, y_train)
            adv_deb.score(X_test, y_test)

            y_pred_adv_deb = adv_deb.predict(X_test)
            writejson(path,y_pred_adv_deb,'adv_deb_pred.json')

            adv_deb.sess_.close() 
        except:
            traceback.print_exc()



        #### ExponentiatedGradientReduction. (Inproc.)

        try:
            consts = ['EqualizedOdds']#['DemographicParity','EqualizedOdds','TruePositiveRateParity','FalsePositiveRateParity','ErrorRateParity']
            for const in consts:
                EGR = ExponentiatedGradientReduction(prot_attr=['SEX_Female','SEX_Male'], estimator=LogisticRegression(solver='liblinear'), constraints = const)
                EGR.fit(X_train, y_train)
                print(const,EGR.score(X_test, y_test))
                y_pred_egr = EGR.predict(X_test)
                writejson(path,y_pred_egr,'egr_pred_'+const+'.json')
        except:
            traceback.print_exc()

        #### GridSearchReduction. (Inproc.)

        try:
            consts = ['EqualizedOdds']#['DemographicParity','EqualizedOdds','TruePositiveRateParity','FalsePositiveRateParity','ErrorRateParity']
            for const in consts:
                GSR = GridSearchReduction(prot_attr=['SEX_Female','SEX_Male'], estimator=LogisticRegression(solver='liblinear'), constraints = const)
                GSR.fit(X_train, y_train)
                print(const,GSR.score(X_test, y_test))
                y_pred_gsr = GSR.predict(X_test)
                writejson(path,y_pred_gsr,'gsr_pred_'+const+'.json')
        except:
            traceback.print_exc()



        X_train_temp = X_train.set_index(['SEX_Male'], drop = False)
        y_train_temp = y_train.copy()
        y_train_temp.index = X_train_temp.index

        X_test_temp = X_test.set_index(['SEX_Male'], drop = False)
        y_test_temp = y_test.copy()
        y_test_temp.index = X_test_temp.index

        #### Cal. Eq.Odds (Postproc.)

        try:
            consts = ['weighted']#['fnr','fpr','weighted']
            for const in consts:
                cal_eq_odds = CalibratedEqualizedOdds(prot_attr='SEX_Male', cost_constraint=const)
                postproc = PostProcessingMeta(estimator=LogisticRegression(solver='liblinear'), postprocessor=cal_eq_odds)
                postproc.fit(X_train_temp, y_train_temp)
                print(accuracy_score(y_test_temp, postproc.predict(X_test_temp)))
                y_pred_caleq = postproc.predict(X_test_temp)
                writejson(path,y_pred_caleq,'caleq_pred_'+const+'.json')
        except:
            traceback.print_exc()

        #### RejectOptionClassifier (Postproc.)

        try:
            consts = ['average_odds']#['statistical_parity', 'average_odds', 'equal_opportunity']
            for const in consts:
                rocv = PostProcessingMeta(LogisticRegression(solver='liblinear'), RejectOptionClassifierCV('SEX_Male', scoring=const))
                rocv.fit(X_train_temp, y_train_temp)
                print(accuracy_score(y_test_temp, rocv.predict(X_test_temp)))
                y_pred_rocv = rocv.predict(X_test_temp)
                writejson(path,y_pred_rocv,'rocv_pred_'+const+'.json')
        except:
            traceback.print_exc()  

        return path,"Done"


In [6]:
# task(path)

In [7]:
for path in paths:
    try:
        task1(path)
    except:
        traceback.print_exc()

data/datasets/credit_tw/synthetic/gender_flip_labels0.1_version6/


2023-02-26 19:44:33.701486: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-26 19:44:35.189514: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46704 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:b1:00.0, compute capability: 8.6
2023-02-26 19:44:35.194250: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46704 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:d9:00.0, compute capability: 8.6
2023-02-26 19:44:35.758027: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not e

EqualizedOdds 0.79
EqualizedOdds 0.8083333333333333
0.7866666666666666
0.7831666666666667
data/datasets/credit_tw/synthetic/gender_flip_labels0.1_version7/


2023-02-26 19:47:01.359735: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46704 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:b1:00.0, compute capability: 8.6
2023-02-26 19:47:01.361158: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46704 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:d9:00.0, compute capability: 8.6


EqualizedOdds 0.7721666666666667
EqualizedOdds 0.7923333333333333
0.7721666666666667
0.7681666666666667
data/datasets/credit_tw/synthetic/gender_flip_labels0.1_version8/


...

EqualizedOdds 0.7736666666666666
EqualizedOdds 0.8008333333333333
0.7738333333333334
0.2803333333333333
data/datasets/credit_tw/synthetic/gender_flip_labels1.0_version7/


2023-02-26 23:30:52.832389: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46704 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:b1:00.0, compute capability: 8.6
2023-02-26 23:30:52.832788: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46704 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:d9:00.0, compute capability: 8.6


EqualizedOdds 0.7778333333333334
EqualizedOdds 0.7943333333333333
0.7775
0.2748333333333333
data/datasets/credit_tw/synthetic/gender_flip_labels1.0_version9/


2023-02-26 23:36:00.809233: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46704 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:b1:00.0, compute capability: 8.6
2023-02-26 23:36:00.810034: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46704 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:d9:00.0, compute capability: 8.6


EqualizedOdds 0.7775
EqualizedOdds 0.802
0.7776666666666666
0.2688333333333333


In [4]:
path = 'data/datasets/law_school/'
# synthfols = glob("data/datasets/law_school/synthetic/*/")
paths = [path]
# paths.extend(synthfols)

In [5]:
def task2(path):
    
    if os.path.exists(path+'preds/rocv_pred_average_odds.json') == False:
        print(path)

        train_df = pd.read_csv(path+'train.csv',header=None)

        train_df.columns = ['zfygpa', 'zgpa', 'DOB_yr', 'weighted_lsat_ugpa', 'cluster_tier',
           'family_income', 'lsat', 'ugpa', 'isPartTime', 'sex', 'race',
           'pass_bar']


        train_df.index = train_df['sex']

        train_df = train_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

        test_df = pd.read_csv(path+'test.csv',header=None)

        test_df.columns = ['zfygpa', 'zgpa', 'DOB_yr', 'weighted_lsat_ugpa', 'cluster_tier',
           'family_income', 'lsat', 'ugpa', 'isPartTime', 'sex', 'race',
           'pass_bar']


        test_df.index = test_df['sex']

        test_df = test_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)



        X_train = train_df.drop(['pass_bar'],axis=1)
        X_test = test_df.drop(['pass_bar'],axis=1)

        y_train = pd.Series(train_df['pass_bar'])
        y_test = pd.Series(test_df['pass_bar'])

        y_train = pd.Series(y_train.factorize(sort=True)[0], index=y_train.index)
        y_test = pd.Series(y_test.factorize(sort=True)[0], index=y_test.index)

        X_merged = pd.concat([X_train,X_test])

        ohe = make_column_transformer(
            (OneHotEncoder(sparse=False), X_merged.dtypes == 'object'),
            remainder='passthrough', verbose_feature_names_out=False)

        X_merged_temp  = pd.DataFrame(ohe.fit_transform(X_merged), columns=ohe.get_feature_names_out(), index=X_merged.index)

        X_train  = pd.DataFrame(ohe.transform(X_train), columns=ohe.get_feature_names_out(), index=X_train.index)
        X_test = pd.DataFrame(ohe.transform(X_test), columns=ohe.get_feature_names_out(), index=X_test.index)


        #### BASELINE
        try:
            y_pred_baseline = LogisticRegression(solver='liblinear').fit(X_train, y_train).predict(X_test)
            writejson(path,y_pred_baseline,'baseline_pred.json')    
        except:
            traceback.print_exc()


        #### Adv. Reweighting (Preproc.)

        try:
            rew = ReweighingMeta(estimator=LogisticRegression(solver='liblinear'),
                         reweigher=Reweighing('sex'))
            params = {'estimator__C': [1, 10]}

            clf = GridSearchCV(rew, params, scoring='accuracy', cv=5)
            clf.fit(X_train, y_train)
            y_pred_advrew = clf.predict(X_test)
            writejson(path,y_pred_advrew,'adv_rew_pred.json')  
        except:
            traceback.print_exc()



        #### LearnedFairRepresentations (Preproc.)

        try:
            LFR = LearnedFairRepresentations(prot_attr='sex')
            LFR.fit(X_train, y_train)

            y_pred_lfr = LFR.predict(X_test)
            writejson(path,y_pred_lfr,'lfr_pred.json')  
        except:
            traceback.print_exc()

        #### Adv Deb. (Inproc.)

        try:
            adv_deb = AdversarialDebiasing(prot_attr='sex')
            adv_deb.fit(X_train, y_train)
            adv_deb.score(X_test, y_test)

            y_pred_adv_deb = adv_deb.predict(X_test)
            writejson(path,y_pred_adv_deb,'adv_deb_pred.json')

            adv_deb.sess_.close() 
        except:
            traceback.print_exc()



        #### ExponentiatedGradientReduction. (Inproc.)

        try:
            consts = ['EqualizedOdds']#['DemographicParity','EqualizedOdds','TruePositiveRateParity','FalsePositiveRateParity','ErrorRateParity']
            for const in consts:
                EGR = ExponentiatedGradientReduction(prot_attr=['sex_Female','sex_Male'], estimator=LogisticRegression(solver='liblinear'), constraints = const)
                EGR.fit(X_train, y_train)
                print(const,EGR.score(X_test, y_test))
                y_pred_egr = EGR.predict(X_test)
                writejson(path,y_pred_egr,'egr_pred_'+const+'.json')
        except:
            traceback.print_exc()

        #### GridSearchReduction. (Inproc.)

        try:
            consts = ['EqualizedOdds']#['DemographicParity','EqualizedOdds','TruePositiveRateParity','FalsePositiveRateParity','ErrorRateParity']
            for const in consts:
                GSR = GridSearchReduction(prot_attr=['sex_Female','sex_Male'], estimator=LogisticRegression(solver='liblinear'), constraints = const)
                GSR.fit(X_train, y_train)
                print(const,GSR.score(X_test, y_test))
                y_pred_gsr = GSR.predict(X_test)
                writejson(path,y_pred_gsr,'gsr_pred_'+const+'.json')
        except:
            traceback.print_exc()



        X_train_temp = X_train.set_index(['sex_Male'], drop = False)
        y_train_temp = y_train.copy()
        y_train_temp.index = X_train_temp.index

        X_test_temp = X_test.set_index(['sex_Male'], drop = False)
        y_test_temp = y_test.copy()
        y_test_temp.index = X_test_temp.index

        #### Cal. Eq.Odds (Postproc.)

        try:
            consts = ['weighted']#['fnr','fpr','weighted']
            for const in consts:
                cal_eq_odds = CalibratedEqualizedOdds(prot_attr='sex_Male', cost_constraint=const)
                postproc = PostProcessingMeta(estimator=LogisticRegression(solver='liblinear'), postprocessor=cal_eq_odds)
                postproc.fit(X_train_temp, y_train_temp)
                print(accuracy_score(y_test_temp, postproc.predict(X_test_temp)))
                y_pred_caleq = postproc.predict(X_test_temp)
                writejson(path,y_pred_caleq,'caleq_pred_'+const+'.json')
        except:
            traceback.print_exc()

        #### RejectOptionClassifier (Postproc.)

        try:
            consts = ['average_odds']#['statistical_parity', 'average_odds', 'equal_opportunity']
            for const in consts:
                rocv = PostProcessingMeta(LogisticRegression(solver='liblinear'), RejectOptionClassifierCV('sex_Male', scoring=const))
                rocv.fit(X_train_temp, y_train_temp)
                print(accuracy_score(y_test_temp, rocv.predict(X_test_temp)))
                y_pred_rocv = rocv.predict(X_test_temp)
                writejson(path,y_pred_rocv,'rocv_pred_'+const+'.json')
        except:
            traceback.print_exc()  

        return path,"Done"


In [6]:
for path in paths:
    try:
        task2(path)
    except:
        traceback.print_exc()

data/datasets/law_school/


2023-02-28 12:46:04.804846: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-28 12:46:06.011477: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46704 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:b1:00.0, compute capability: 8.6
2023-02-28 12:46:06.012136: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46704 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:d9:00.0, compute capability: 8.6
2023-02-28 12:46:06.447503: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not e

EqualizedOdds 0.8290340802108831
EqualizedOdds 0.822632272641687
0.8288457917529656
0.8092637921295425
